In [1]:
import tensorflow as tf
import scipy.io 
import matplotlib.pyplot as plt
import cv2
import keras
from glob import glob
import numpy as np
from tqdm import tqdm
import os
from PIL import Image
import pandas as pd
import cv2

from sklearn.model_selection import KFold
# from keras.preprocessing.image import ImageDataGenerator

# import keras_metrics

from keras.applications import mobilenet, resnet50 #, vgg16, inception_v3, resnet50, 
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, History

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

import logging
# logging.getLogger().setLevel(logging.DEBUG)


from scikitplot.metrics import plot_roc_curve

Using TensorFlow backend.


In [2]:
keras.__version__

'2.2.4'

## Params

In [3]:
# all_data_dir = 'E:\\Work/PathoBarIlan/Shlomi2018/'# '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018'
all_data_dir = '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018'

is_relative_path_csv = False
seed = 4221

pos_name_init = 'Cancer'
neg_name_init = 'Normal'

use_rgb = False # True=rgb, False=spectral
if use_rgb:
    file_ext = '.png'
else:
    file_ext = '.npy'
    
window_size = (200, 200)
shift = (100, 100)

In [4]:
w,h = window_size
if use_rgb:
    input_shape = (w,h,3)
else:
    input_shape = (w,h,40)
batch_size = 16

## utils

In [5]:
def read_slide(path):
    mat = scipy.io.loadmat(path)
    spectral = mat["Spec"]
    rgb = mat["Section"]
    shape = rgb.shape
    
    return spectral, rgb

In [6]:
def create_batch_of_crops_from_slide(img, window_size, shift, vis_flag=False):
    crops = []

    n_iter_x = (img.shape[1]-window_size[0])//shift[0] + 1

    n_iter_y = (img.shape[0]-window_size[1])//shift[1] + 1

#     n_iter_x, n_iter_y

    for i in range(n_iter_x):
        for j in range(n_iter_y):
            init_y = i*shift[0]
            init_x = j*shift[1]
        
            crops.append(img[init_x:init_x+window_size[0], init_y:init_y+window_size[1], :])
    if vis_flag:
        visualize_batch_of_crops(crops, n_iter_y, n_iter_x)
    return crops

In [7]:
def visualize_batch_of_crops(crops, n_iter_y, n_iter_x):
    fig, axes = plt.subplots(n_iter_y, n_iter_x, figsize=(5, 5), gridspec_kw = {'wspace':0, 'hspace':0})

    for i in range(n_iter_x):
        for j in range(n_iter_y):
            axes[j, i].imshow(crops[i*n_iter_y + j])
            axes[j, i].axis('off')
            axes[j, i].set_aspect('equal')
    plt.show()

In [8]:
def create_crops_from_fileslist(fileslist, window_size, shift):
    rgb_crops = []
    spectral_crops = []
    labels = []

    for file in tqdm(fileslist):
#         file_name = os.path.basename(file)
#         print('Saving crops for file {} ...'.format(file_name))
#         print(file)
        spectral, rgb = read_slide(file)
        spectral_crops = create_batch_of_crops_from_slide(spectral, window_size=window_size, shift=shift)
        rgb_crops = create_batch_of_crops_from_slide(rgb, window_size=window_size, shift=shift)
        save_dir = file.replace('.mat', '_win{}-{}_shift{}-{}'.format(window_size[0], window_size[1], shift[0], shift[1]))
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        for idx, (im_np, spec_np) in enumerate(zip(rgb_crops, spectral_crops)):
            im = Image.fromarray(im_np)
            im.save(os.path.join(save_dir, '{:05}.png'.format(idx)))
            np.save(os.path.join(save_dir, '{:05}.npy'.format(idx)), spec_np)


In [9]:
def create_crops_from_dir(dir_path, window_size, shift):
    print('Saving crops for slides in dir: {}'.format(dir_path))
    fileslist = glob(dir_path + '/*.mat')
    create_crops_from_fileslist(fileslist, window_size, shift)

In [10]:
def create_csv_for_folder(data_dir, ext):
    if ext[0] == '.':
        ext = ext[1:]
    data_df = pd.DataFrame(columns=['filename', 'label'])
    files = glob(os.path.join(data_dir,'*', '*.{}'.format(ext)))
    files = [file for file in files if "Mixed" not in file]
#     print(data_dir+'/*/*.{}'.format(ext))
    
    init_len = len(data_dir)
    delete_folder = all_data_dir
    if not is_relative_path_csv:
        delete_folder = '/'
    if not delete_folder[-1] == '/':
        delete_folder += '/'
    files = [file.replace(delete_folder, '/') for file in files]
#     print(files)
    labels = [1 if pos_name_init in file else 0 for file in files]
#     print(labels)
    data_df['filename'] = files
    data_df['label'] = labels
#     data_df.to_csv(os.path.join(data_dir, os.path.basename(data_dir)+'.csv'), index=False)
#     print('Created CSV successfully for folder {}'.format(data_dir))
    
    return data_df    

In [11]:
slides = glob(os.path.join(all_data_dir, "*/"))
slides

['/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case10/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case11/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case12/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case14/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case16/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case16b/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case17/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case18/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case19484/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case8/']

In [12]:
skf = KFold(n_splits=5, shuffle=True, random_state=seed)

train_slides_all = []
test_slides_all = []
val_slides_all = []

for train_index, test_index in skf.split(np.arange(len(slides)).T, np.arange(len(slides)).T):
    print("TRAIN:", train_index, "TEST:", test_index)
    train_slides_all.append(train_index)
    val_slides_all.append([test_index[0]])
    test_slides_all.append([test_index[1]])

TRAIN: [2 3 4 5 6 7 8 9] TEST: [0 1]
TRAIN: [0 1 2 3 5 7 8 9] TEST: [4 6]
TRAIN: [0 1 3 4 5 6 7 8] TEST: [2 9]
TRAIN: [0 1 2 4 6 7 8 9] TEST: [3 5]
TRAIN: [0 1 2 3 4 5 6 9] TEST: [7 8]


In [13]:
k_idx = 1 # take one of the K-Folds

train_index = train_slides_all[k_idx]
val_index = val_slides_all[k_idx]
test_index = test_slides_all[k_idx]

train_index, val_index, test_index

(array([0, 1, 2, 3, 5, 7, 8, 9]), [4], [6])

In [14]:
def get_dfs_for_indices(slides, index_list):
    dfs = []
    for slide in np.array(slides)[index_list]:
        data_dir = slide
        dfs.append(create_csv_for_folder(data_dir, file_ext))
    df = pd.concat(dfs, ignore_index=True)
    df = df.sample(frac=1, random_state=seed)  # frac=1 is same as shuffling df.
    return df

In [15]:
df_train = get_dfs_for_indices(slides, train_index)
df_test = get_dfs_for_indices(slides, test_index)
df_val = get_dfs_for_indices(slides, val_index)

In [16]:
pd.options.display.max_colwidth = 150

In [17]:
print(len(df_train.index.values))
print(len(set(df_train.index.values)))

print(len(df_train.columns.values))
print(len(set(df_train.columns.values)))

2598
2598
2
2


In [18]:
assert len(set(df_train.label.values)) == 2 and len(set(df_val.label.values)) == 2 and len(set(df_test.label.values)) == 2  

In [ ]:
def batch_norm(x):
#     print("x.shape:", x.shape)
    maxi = x.max(axis=1).max(axis=1)
#     print("maxi.shape:", maxi.shape)
    maxi = np.repeat(maxi[:, np.newaxis, : ], window_size[0], axis=1)
    maxi = np.repeat(maxi[:, np.newaxis, : ], window_size[1], axis=1)
    return x/maxi


def generator_from_df(df, batch_size, shuffle=True): 
    
    n_batches = df.shape[0]//batch_size
    while True:
        if shuffle:
            df_tmp = df.copy().sample(frac=1)  # frac=1 is same as shuffling df.
        else:
            df_tmp = df
        
        for i in range(n_batches):
            sub = df_tmp.iloc[batch_size*i:batch_size*(i+1)]
            if use_rgb:
                X = [img_to_array(load_img(f, target_size=input_shape)) for f in sub.filename]
            else:
                X = [np.load(f) for f in sub.filename]
                
            X = batch_norm(np.stack(X))
            logging.debug("from file {}\nto file {}".format(sub.iloc[0].filename, sub.iloc[-1].filename))

            Y = sub.label.values
            Y = to_categorical(Y, num_classes=2)
            # Simple model, one input, one output.
            
            yield X, Y

####### copying generator_from_df:
https://gist.github.com/timehaven/257eef5b0e2d9e2625a9eb812ca2226b#file-akmtdfgen-py

In [ ]:
train_generator = generator_from_df(df_train, batch_size)
val_generator = generator_from_df(df_val, batch_size)
test_generator = generator_from_df(df_test, batch_size, shuffle=False)

In [ ]:
# input_shape = train_generator.image_shape
mobilenet_model = mobilenet.MobileNet(include_top=True, weights=None, input_shape=input_shape, classes=2, dropout=0.2)
# mobilenet_model = resnet50.ResNet50(include_top=True, weights=None, input_shape=input_shape, classes=2)

/media/leetwito/Windows/Users/leetw/PycharmProjects/venvs/keras/lib/python3.5/site-packages/keras_applications/imagenet_utils.py:279: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 40 input channels.
  str(input_shape[-1]) + ' input channels.')


In [ ]:
mobilenet_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 40)      0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 201, 201, 40)      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 100, 100, 32)      11520     
_________________________________________________________________
conv1_bn (BatchNormalization (None, 100, 100, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 100, 100, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 100, 100, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 100, 100, 32)      128       
__________

In [ ]:
optimizer = Adam(lr=1e-2) # 1e-3
mobilenet_model.compile(loss="binary_crossentropy", optimizer=optimizer) #  binary_crossentropy , categorical_crossentropy
# history = History()
lrReduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=4, verbose=1, min_lr=1e-6)
if use_rgb:
    chkpnt = ModelCheckpoint("my_models/model_rgb_weights_epoch{epoch:02d}-val_loss{val_loss:.3f}"+"-seed{}-k_idx={}.hdf5".format(seed, k_idx), save_best_only=False) # -train_loss{history.History()[loss][-1]:.2f}
else:
    chkpnt = ModelCheckpoint("my_models/model_spec_weights_epoch{epoch:02d}-val_loss{val_loss:.3f}"+"-seed{}-k_idx={}.hdf5".format(seed, k_idx), save_best_only=False) # -train_loss{history.History()[loss][-1]:.2f}
num_of_epochs = 100

In [ ]:
STEP_SIZE_TRAIN=len(df_train)//batch_size
STEP_SIZE_VALID=len(df_val)//batch_size
# print(STEP_SIZE_VALID)
history = mobilenet_model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=num_of_epochs, callbacks=[lrReduce, chkpnt], shuffle=False) # chkpnt

Epoch 1/100
162/162 [==============================] - 692s 4s/step - loss: 0.1908 - val_loss: 4.2330
Epoch 2/100
162/162 [==============================] - 705s 4s/step - loss: 0.1122 - val_loss: 0.0050
Epoch 3/100
162/162 [==============================] - 705s 4s/step - loss: 0.0844 - val_loss: 6.1952
Epoch 4/100
162/162 [==============================] - 710s 4s/step - loss: 0.0994 - val_loss: 6.4738
Epoch 5/100
162/162 [==============================] - 710s 4s/step - loss: 0.0781 - val_loss: 0.8150
Epoch 6/100
162/162 [==============================] - 713s 4s/step - loss: 0.0817 - val_loss: 4.3206

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0029999999329447745.
Epoch 7/100
162/162 [==============================] - 698s 4s/step - loss: 0.0563 - val_loss: 0.3857
Epoch 8/100
162/162 [==============================] - 718s 4s/step - loss: 0.0309 - val_loss: 0.0430
Epoch 9/100
162/162 [==============================] - 713s 4s/step - loss: 0.0187 - val_loss: 0.0154
E

In [ ]:
x, y = next(test_generator)
y_proba = mobilenet_model.predict(x)
# mobilenet_model.predict(next())

In [ ]:
print(y)

In [ ]:
mobilenet_model.evaluate(x, y)

In [ ]:
y_proba.argmax(axis=1)==y.argmax(axis=1)

In [ ]:
# STEP_SIZE_TEST=len(df_test)//batch_size
# mobilenet_model.evaluate_generator(train_generator, steps=1)
mobilenet_model.evaluate(xxx, yyy)

In [ ]:
# y_pred = mobilenet_model.predict_generator(train_generator, steps=1)
y_pred = mobilenet_model.predict(xx)
y_pred

In [ ]:
y_pred.argmax(axis=1), y_train.values[:, 1].astype(int)

In [ ]:
y_test = df_test['label'][:len(y_pred)].values

In [ ]:
plot_roc_curve(y_test, y_pred)
plt.show()

In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_pred.shape

In [ ]:
y_pred.sum()

In [ ]:
y_test.sum()

In [ ]:
(y_pred==y_test).sum()/220